In [1]:
import openai
import os
import json

# read config file from root directory
config_path = '../config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = config['OPENAI_API_KEY']

# 1. Chat Format Introduction

## 1.1 Response returned by model:

In [2]:
#### The chat completion object example
example = {
  "id": "chatcmpl-123",                       # A unique identifier for the chat completion.
  "object": "chat.completion",
  "created": 1677652288,
  "model": "gpt-3.5-turbo-0613",
  "choices": [{
    "index": 0,
    "message": {
      "role": "assistant",                    # The role of the author of this message.
      "content": "\n\nHello there, how may I assist you today?",
    },
    "finish_reason": "stop"                   # The reason why the chat stopped.
  }],
  "usage": {                                  # Usage statistics for the completion request.
    "prompt_tokens": 9,
    "completion_tokens": 12,
    "total_tokens": 21
  }
}

#### The chat completion chunk object example
# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"role":"assistant","content":""},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"Hello"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"!"},"finish_reason":null}]}

# ....

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":" today"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{"content":"?"},"finish_reason":null}]}

# {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0613","choices":[{"index":0,"delta":{},"finish_reason":"stop"}]}

## 1.2 Create chat completion (request and response):
Full information and parameters: [Click Here](https://platform.openai.com/docs/api-reference/chat/create)

In [3]:
# No streaming
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4",
  # temperature=0.9,  # between 0 and 2, higher -> more diversity
  # top_p=1,         # between 0 and 1, higher -> less diversity. Altering this or temperature but not both is recommended
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! What's your model? gpt-3.5 or gpt-4?"},
  ]
)

# print(completion.choices[0].message)
completion

<OpenAIObject chat.completion id=chatcmpl-885yToHojWdXeSF1U2JPzKKg6qUhQ at 0x2a35d5d31d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "As an AI developed by OpenAI, my underlying model is GPT-3. As of the current moment, a GPT-4 iteration has not been released.",
        "role": "assistant"
      }
    }
  ],
  "created": 1696940281,
  "id": "chatcmpl-885yToHojWdXeSF1U2JPzKKg6qUhQ",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 34,
    "prompt_tokens": 36,
    "total_tokens": 70
  }
}

In [4]:
# Streaming
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta)

{
  "content": "",
  "role": "assistant"
}
{
  "content": "Hello"
}
{
  "content": "!"
}
{
  "content": " How"
}
{
  "content": " can"
}
{
  "content": " I"
}
{
  "content": " assist"
}
{
  "content": " you"
}
{
  "content": " today"
}
{
  "content": "?"
}
{}


In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

functions = [ 
  {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location", # used by the model to choose when and how to call the function.
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
    },
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  functions=functions,
  function_call="auto",  # auto is default, but we'll be explicit. "none" means the model will not call a function and instead generates a message. "auto" means the model can pick between generating a message or calling a function. Specifying a particular function via {"name": "my_function"} forces the model to call that function
)

print(completion)


{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1696940295,
  "id": "chatcmpl-885yhzWPQMglofgypg9S1FFgevTjW",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 83,
    "total_tokens": 101
  }
}


# 2. Advanced usage

## 2.1 Example of function call:
> Function calling allows you to more reliably get structured data back from the model. For example, you can:
> - Create chatbots that answer questions by calling external APIs (e.g. like ChatGPT Plugins)
>     - e.g. define functions like send_email(to: string, body: string), or get_current_weather(location: > string, unit: 'celsius' | 'fahrenheit')
> - Convert natural language into API calls
>     - e.g. convert "Who are my top customers?" to get_customers(min_revenue: int, created_before: string, > limit: int) and call your internal API
> - Extract structured data from text
>     - e.g. define a function called extract_data(name: string, birthday: string), or sql_query(query: string)
> 
> ...and much more!

> The basic sequence of steps for function calling is as follows:
> 1. Call the model with the user query and a set of functions defined in the functions parameter.
> 2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
> 3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
> 4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

In [6]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation())

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees.",
        "role": "assistant"
      }
    }
  ],
  "created": 1696942437,
  "id": "chatcmpl-886XFpNzJDkwvCEHQdwsfQkphqeoo",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 72,
    "total_tokens": 89
  }
}
